In [84]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import pandas as pd
import os

### Data exploration
['AS14.01', 'AS14.02', 'AS14.03', 'AS14.05', 'AS14.06', 'AS14.07',
       'AS14.08', 'AS14.09', 'AS14.12', 'AS14.13', 'AS14.14', 'AS14.15',
       'AS14.16', 'AS14.17', 'AS14.19', 'AS14.20', 'AS14.23', 'AS14.24',
       'AS14.25', 'AS14.26', 'AS14.27', 'AS14.28', 'AS14.29', 'AS14.30',
       'AS14.31', 'AS14.32', 'AS14.33']

In [85]:
df = pd.read_csv('dataset_mood_smartphone.csv')

In [86]:
df["id_num"] = df["id"].apply(lambda x: int(x.split(".")[1]))
print(df["id_num"].unique())
df["id"].unique()



[ 1  2  3  5  6  7  8  9 12 13 14 15 16 17 19 20 23 24 25 26 27 28 29 30
 31 32 33]


array(['AS14.01', 'AS14.02', 'AS14.03', 'AS14.05', 'AS14.06', 'AS14.07',
       'AS14.08', 'AS14.09', 'AS14.12', 'AS14.13', 'AS14.14', 'AS14.15',
       'AS14.16', 'AS14.17', 'AS14.19', 'AS14.20', 'AS14.23', 'AS14.24',
       'AS14.25', 'AS14.26', 'AS14.27', 'AS14.28', 'AS14.29', 'AS14.30',
       'AS14.31', 'AS14.32', 'AS14.33'], dtype=object)

In [87]:
# convert time to datetime
df["time"] = pd.to_datetime(df["time"])


In [88]:
for person in df["id_num"].unique():
    print("PERSON", person)
    # df_person = df[df["id_num"] == person]
    # print(df_person["variable"].unique())
    # print(df_person["variable"].value_counts())
    # print(df_person["value"].describe())
    # print(df_person["time"].describe())
    # print(df_person["time"].max() - df_person["time"].min())
    # print(df_person["time"].max())
    # print(df_person["time"].min())

    for var in df_person["variable"].unique():
        print(var)
        df_var = df_person[df_person["variable"] == var]
        print(df_var["value"].describe())
        print(df_var["time"].describe())
        print(df_var["time"].max() - df_var["time"].min())
        print(df_var["time"].max())
        print(df_var["time"].min())


PERSON 1
mood
count    192.000000
mean       6.562500
std        1.481562
min        2.000000
25%        6.000000
50%        7.000000
75%        7.000000
max       10.000000
Name: value, dtype: float64
count                              192
mean     2014-05-09 04:52:48.750000128
min                2014-04-16 15:00:00
25%                2014-04-26 18:00:00
50%                2014-05-10 10:30:00
75%                2014-05-21 02:15:00
max                2014-05-31 12:00:00
Name: time, dtype: object
44 days 21:00:00
2014-05-31 12:00:00
2014-04-16 15:00:00
circumplex.arousal
count    191.000000
mean      -0.382199
std        1.246256
min       -2.000000
25%       -2.000000
50%        0.000000
75%        1.000000
max        2.000000
Name: value, dtype: float64
count                              192
mean     2014-05-09 04:52:48.750000128
min                2014-04-16 15:00:00
25%                2014-04-26 18:00:00
50%                2014-05-10 10:30:00
75%                2014-05-21 02:15:00
m

In [93]:
df_person_1 = df[df["id_num"] == 1].copy()

# df_person_1["time"].nunique()

# count which number occurs twice

# split the values into variable columns with unique time values

# count unique days in time
# print(df_person_1["time"].dt.date.nunique())

# rename time into date-time
df_person_1 = df_person_1.rename(columns={"time": "datetime"})
df_person_1["date"] = df_person_1["datetime"].dt.date
df_person_1["hour"] = df_person_1["datetime"].dt.hour
df_person_1["minute"] = df_person_1["datetime"].dt.minute
df_person_1



,Unnamed: 0,id,datetime,variable,value,id_num,date,hour,minute
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.000,1,2014-02-26,13,0
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.000,1,2014-02-26,15,0
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.000,1,2014-02-26,18,0
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.000,1,2014-02-26,21,0
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.000,1,2014-02-27,9,0
...,...,...,...,...,...,...,...,...,...
374684,2348657,AS14.01,2014-05-05 00:15:26.313,appCat.utilities,164.567,1,2014-05-05,0,15
374685,2348659,AS14.01,2014-05-05 00:18:14.953,appCat.utilities,48.138,1,2014-05-05,0,18
374686,2348660,AS14.01,2014-05-05 00:19:42.322,appCat.utilities,48.156,1,2014-05-05,0,19
374687,2348691,AS14.01,2014-05-05 09:41:03.888,appCat.utilities,3.470,1,2014-05-05,9,41


None
